In [1]:
# Dependencies

# Import Splinter
from splinter import Browser
# Import BeautifulSoup
from bs4 import BeautifulSoup as soup

import requests
import os
import pandas as pd
from pandas import DataFrame
import re

from webdriver_manager.chrome import ChromeDriverManager

In [55]:
# Python code to convert string to list
  
def Convert(string):
    li = list(string.split("<td>"))
    return li

In [2]:
executable_path = {'executable_path': ChromeDriverManager().install()}
browser = Browser('chrome', **executable_path, headless=False)



====== WebDriver manager ======
Current google-chrome version is 91.0.4472
Get LATEST driver version for 91.0.4472
Driver [C:\Users\paul\.wdm\drivers\chromedriver\win32\91.0.4472.101\chromedriver.exe] found in cache


In [3]:
# Function to remove tags
def remove_tags(html):
  
    # parse html content
    soup = BeautifulSoup(html, "html.parser")
  
    for data in soup(['style', 'script']):
        # Remove tags
        data.decompose()
  
    # return data by retrieving the tag content
    return ' '.join(soup.stripped_strings)

In [4]:
## base_url
url = 'https://www.olympedia.org/athletes/93860'

## site statistics
site_statistics = 'https://www.olympedia.org/statistics/olympedia'

In [5]:
browser.visit(url)
# Optional delay for loading the page
browser.is_element_present_by_css('table.biodata', wait_time=1)

True

In [6]:
html = browser.html
news_soup = soup(html, 'html.parser')

In [7]:
biodata = news_soup.select_one('table.biodata')
bio_column = biodata.findChildren(['th'])
bio_column = [str(i).replace( r"<th>", '').replace( r"</th>", '') for i in bio_column]
#bio_column = [str(i).replace( r"</th>", '') for i in bio_column]
bio_column

['Type',
 'Sex',
 'Full name',
 'Used name',
 'Nick/petnames',
 'Born',
 'Measurements',
 'Affiliations',
 'NOC']

In [8]:
#biodate = news_soup.select_one('table.biodata')
#biodate_columns=DataFrame(biodate.findChildren(['th']))
#biodate_columns

In [9]:
#biodate.findChildren(['td'])#.replace('\<\/*td\>','',regex=True)
bio_data = biodata.findChildren(['td'])
bio_data = [str(i).replace( r'<td>', '') for i in bio_data]
bio_data = [str(i).replace( r'</td>', '') for i in bio_data]
bio_data = [str(i).replace( r'•', ' ') for i in bio_data]

#bio_data = [str(i).replace( r'<a href="/affiliations/', '') for i in bio_data]
#bio_data = [str(i).replace( r'<img src="/images/flags/', '') for i in bio_data]
#bio_data = [str(i).replace( r'">.+</a>', '') for i in bio_data]
#bio_data = [str(i).replace( r'.png.+', '') for i in bio_data]
bio_data

['Competed in Olympic Games',
 'Male',
 'Michael Fred Phelps, II',
 'Michael Phelps',
 'The Baltimore Bullet',
 '30 June 1985 in <a href="/place_names/4423">Baltimore, Maryland (USA)</a>',
 '193 cm / 91 kg',
 '<a href="/affiliations/6636">NBAC, Baltimore (USA)</a> / <a href="/affiliations/3411">Michigan Wolverines, Ann Arbor (USA)</a>',
 '<img src="/images/flags/USA.png" style="padding-right: 2px; vertical-align: middle"/> <a href="/countries/USA">United States</a>']

In [10]:
biodata_values=DataFrame(biodata.findChildren(['td']))
biodata_values


D:\ProgramData\Anaconda3\envs\CUVIRTDATA2021\lib\site-packages\pandas\core\internals\construction.py:309: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  values = np.array([convert(v) for v in values])


,0
0,[Competed in Olympic Games]
1,[Male]
2,"[Michael Fred•Phelps, II]"
3,[Michael•Phelps]
4,[The Baltimore Bullet]
5,"[30 June 1985 in , [Baltimore, Maryland (USA)]]"
6,[193 cm / 91 kg]
7,"[[NBAC, Baltimore (USA)], / , [Michigan Wolve..."
8,"[[], , [United States]]"


In [11]:
df = pd.DataFrame(columns=bio_column)
a_series = pd.Series(bio_data, index = df.columns)
df = df.append(a_series, ignore_index=True)
df

,Type,Sex,Full name,Used name,Nick/petnames,Born,Measurements,Affiliations,NOC
0,Competed in Olympic Games,Male,"Michael Fred Phelps, II",Michael Phelps,The Baltimore Bullet,"30 June 1985 in <a href=""/place_names/4423"">Ba...",193 cm / 91 kg,"<a href=""/affiliations/6636"">NBAC, Baltimore (...","<img src=""/images/flags/USA.png"" style=""paddin..."


In [18]:
bioResults_column = news_soup.select_one('table.table')
bioResults_column = bioResults_column.findChildren(['th'])
bioResults_column = [str(i).replace( r"<th>", '').replace( r"</th>", '')  for i in bioResults_column]
bioResults_column

['Games', 'Discipline (Sport) / Event', 'NOC / Team', 'Pos', 'Medal', 'As', '']

In [28]:
len(bioResults_column)

7

In [46]:
bioResults_data = news_soup.select_one('table.table')
bioResults_data = bioResults_data.findAll(['tr'])
len(bioResults_data)

36

In [61]:
bioResults_data = news_soup.select_one('table.table')
bioResults_data = bioResults_data.findAll(['tr'])[1:]
bioResults_data = [str(i).replace( '\n', '').
                   replace( r"<tr>", '').replace( r"</tr>", '').
                   replace( r"</td>", '')
                   for i in bioResults_data]
bioResults_data

['<tr class="active"><td><a href="/editions/25">2000 Summer Olympics</a><td><a href="/sports/SWI">Swimming</a> (<a href="/sport_groups/AQ">Aquatics</a>)<td><a href="/countries/USA"><img src="/images/flags/USA.png" style="padding-right: 2px; vertical-align: middle"/>USA</a><td><td><td>Michael Phelps<td>',
 '<td><td><a href="/results/8466">200 metres Butterfly, Men</a><small>(Olympic)</small><td><td>5<td><td><td>',
 '<tr class="active"><td><a href="/editions/26">2004 Summer Olympics</a><td><a href="/sports/SWI">Swimming</a> (<a href="/sport_groups/AQ">Aquatics</a>)<td><a href="/countries/USA"><img src="/images/flags/USA.png" style="padding-right: 2px; vertical-align: middle"/>USA</a><td><td><td>Michael Phelps<td>',
 '<td><td><a href="/results/8772">200 metres Freestyle, Men</a><small>(Olympic)</small><td><td>3<td><span class="Bronze">Bronze</span><td><td>',
 '<td><td><a href="/results/8898">4 × 100 metres Freestyle Relay, Men</a><small>(Olympic)</small><td>United States<td>3<td><span cla

In [64]:
for x in bioResults_data:
    #set active class as prefix
    #set rest as dataframe insert
  if x.startswith('<tr class="active">'):
    print(Convert(x))
#  print("\n")

['<tr class="active">', '<a href="/editions/25">2000 Summer Olympics</a>', '<a href="/sports/SWI">Swimming</a> (<a href="/sport_groups/AQ">Aquatics</a>)', '<a href="/countries/USA"><img src="/images/flags/USA.png" style="padding-right: 2px; vertical-align: middle"/>USA</a>', '', '', 'Michael Phelps', '']
['<tr class="active">', '<a href="/editions/26">2004 Summer Olympics</a>', '<a href="/sports/SWI">Swimming</a> (<a href="/sport_groups/AQ">Aquatics</a>)', '<a href="/countries/USA"><img src="/images/flags/USA.png" style="padding-right: 2px; vertical-align: middle"/>USA</a>', '', '', 'Michael Phelps', '']
['<tr class="active">', '<a href="/editions/53">2008 Summer Olympics</a>', '<a href="/sports/SWI">Swimming</a> (<a href="/sport_groups/AQ">Aquatics</a>)', '<a href="/countries/USA"><img src="/images/flags/USA.png" style="padding-right: 2px; vertical-align: middle"/>USA</a>', '', '', 'Michael Phelps', '']
['<tr class="active">', '<a href="/editions/54">2012 Summer Olympics</a>', '<a hre

36

In [20]:
browser.quit()